## Import

In [5]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import importlib
import NAS 
import netparser
import numpy as np
import os
import pandas as pd
import q_learner
import state_string_utils as stringutils
import state_enumerator as stateenum
from tensorflow import keras
import time
from tqdm import tqdm

## Loading Data

In [10]:
# Window length between features: number of days
feat_window = 90

# Window length between performance: number of years
pm_window = 3
lb_window = int(3 * pm_window * 365.25) + 1

# Window length between training samples: number of days
sample_window = 30

# Test period start date
test_start_date = '2020-06-30'

# Filepath of dataset
filepath = '/Users/kz_ke/Documents/Masters/Classes/DL/WM-SecuritySelection-main/data/MF_LargeCap_ExcessReturn_3Y.parquet'

In [11]:
def prepare_data_for_er_ari(filepath, label_window=pm_window):
    ''' Cleans data: 
            1.Dropping rows with missing value; 
            2.Dropping columns without data;
            3.Removing columns with available data period shorter than pm_window
        Then sets the data after the performance window as labels
        Returns dictionary of cleaned data and dictionary of labels
    '''
    er_ari_df = pd.read_parquet(filepath)
    

    data_dict = {ticker: er_ari_df[ticker].dropna() for ticker in er_ari_df.columns}
    tickers_to_remove = []
    
    label_dict = {}
    for ticker, series in tqdm(data_dict.items()):
        if series.isna().sum() == series.shape[0]:
            tickers_to_remove += [ticker]
            continue

        last_date = series.index[-1] - relativedelta(years=pm_window)
        if last_date <= series.index[0]:
            tickers_to_remove.append(ticker)
            continue

        index = series.loc[:series.index[-1] - relativedelta(years=pm_window)].index
        label_dict[ticker] = pd.Series([
            series[date + relativedelta(years=pm_window)] for date in index
        ], index=index)
        
    _ = [data_dict.pop(ticker) for ticker in tickers_to_remove]
    
    return data_dict, label_dict

In [12]:
%%time
data_dict, label_dict = prepare_data_for_er_ari(filepath)

100%|██████████████████████████████████████████████████████████████████████████████| 1330/1330 [05:26<00:00,  4.07it/s]

Wall time: 5min 28s


In [13]:
%%time
'''Looping through the dictionary of data, lets the data before checkpoint to be training data and the forward to be testing data. 
Gets list of data and labels according to lb_window and feat_window. '''

tickers = list(data_dict.keys())

train_data = []
train_labels = []

test_data = []
test_labels = []

# test start date
checkpoint = datetime.strptime(test_start_date, '%Y-%m-%d') - relativedelta(years=pm_window)

for ticker in tqdm(tickers):    
    label = label_dict[ticker]
    if label.shape[0] == 0:
        continue
    ts = data_dict[ticker].loc[:label.index[-1]]

    indices = [np.arange(i, i+lb_window, feat_window) for i in range(0, ts.shape[0] - lb_window + 1, sample_window)]
    
    temp_data = np.array([ts.iloc[sub_indices].values for sub_indices in indices])
    if temp_data.shape[0] == 0:
        continue
    temp_labels = np.array([label.loc[ts.index[sub_indices[-1]]] for sub_indices in indices])
    
    train_indices = [idx for idx in range(temp_data.shape[0]) if ts.index[indices[idx][-1]] <= checkpoint]
    test_indices = [idx for idx in range(temp_data.shape[0]) if ts.index[indices[idx][-1]] > checkpoint]
    
    train_data += [temp_data[train_indices]] 
    train_labels += [temp_labels[train_indices]]
    
    test_data += [temp_data[test_indices]] 
    test_labels += [temp_labels[test_indices]]



100%|██████████████████████████████████████████████████████████████████████████████| 1130/1130 [00:13<00:00, 84.10it/s]

Wall time: 13.4 s


In [14]:
%%time
'''
Reshapes and gets x_train, x_test, y_train, y_test
Then uses the first 90% of train data to be the actual training dataset, and the final 10% to be the validation dataset.
'''
x_train = np.concatenate(train_data)[:, :, np.newaxis]
y_train = np.concatenate(train_labels)[:, np.newaxis]
x_test = np.concatenate(test_data)[:, :, np.newaxis]
y_test = np.concatenate(test_labels)

train_split = int(len(x_train) * 0.9)

x_train, x_val, y_train, y_val = x_train[:train_split], x_train[train_split:], y_train[:train_split], y_train[train_split:]

Wall time: 10.9 ms


## Running NAS

In [36]:
'''Initialize directory'''
path = '/Users/kz_ke/Documents/Masters/Classes/DL/AutoML3/mylogs'
isExist = os.path.exists(path)
if not isExist:
    os.makedirs(path)
    print("The new directory is created!")

In [37]:
'''Import necessary parameters and the module that generates networks'''
_model = __import__('models.AutoML',
                    globals(),
                    locals(),
                    ['state_space_parameters', 'hyper_parameters'], 
                    0)

In [65]:
'''
At each iteration, gets loss, RMSE and accuracy, then stores the trained network to replay database for future use
Training stops when the monitored 'val_loss' has stopped improving.
'''

for epsilon, iterations in _model.state_space_parameters.epsilon_schedule:

    factory = NAS.NAS('mylogs',
                  _model.state_space_parameters,
                  _model.hyper_parameters,
                  epsilon = epsilon)


    for runtimes in range(iterations):
        net, i = factory.generate_new_network()
        print(net)
        p= netparser.parse('net', net)
        newnet = netparser.parse_network_structure(p)
        model = keras.Sequential(newnet)
        
        callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights = True)
        model.compile(
                    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                    loss='mean_squared_error',
                    metrics=[keras.metrics.RootMeanSquaredError()]
                )
        
        history = model.fit(x_train, y_train, batch_size = 40, epochs =100, callbacks=[callback], validation_data=(x_val, y_val))
        
        bestval = model.evaluate(x_val, y_val)[1]
        
        factory.incorporate_trained_net(net, bestval, epsilon, [i])

[RNN(30,sigmoid), TERMINATE]
328/328 [==============================] - 1s 3ms/step - loss: 0.0048 - root_mean_squared_error: 0.0695
Incorporated net, acc: 0.069538, net: [RNN(30,sigmoid), TERMINATE]
[LSTM(30,sigmoid), D(0.300000), LSTM(20,leaky_relu), FC(30, tanh), D(0.400000), TERMINATE]
328/328 [==============================] - 4s 10ms/step - loss: 0.0053 - root_mean_squared_error: 0.0725
Incorporated net, acc: 0.072472, net: [LSTM(30,sigmoid), D(0.300000), LSTM(20,leaky_relu), FC(30, tanh), D(0.400000), TERMINATE]
[RNN(30,leaky_relu), LSTM(30,tanh), TERMINATE]
328/328 [==============================] - 3s 9ms/step - loss: 0.0043 - root_mean_squared_error: 0.0659
Incorporated net, acc: 0.065860, net: [RNN(30,leaky_relu), LSTM(30,tanh), TERMINATE]
[LSTM(20,relu), FC(30, relu), D(0.300000), TERMINATE]
328/328 [==============================] - 2s 6ms/step - loss: 0.0045 - root_mean_squared_error: 0.0669
Incorporated net, acc: 0.066874, net: [LSTM(20,relu), FC(30, relu), D(0.300000), 

328/328 [==============================] - 2s 6ms/step - loss: 0.0050 - root_mean_squared_error: 0.0709
Incorporated net, acc: 0.070916, net: [GRU(10,sigmoid), FC(10, leaky_relu), TERMINATE]
[LSTM(10,leaky_relu), D(0.400000), TERMINATE]
328/328 [==============================] - 2s 5ms/step - loss: 0.0049 - root_mean_squared_error: 0.0698
Incorporated net, acc: 0.069819, net: [LSTM(10,leaky_relu), D(0.400000), TERMINATE]
[BILSTM(10,leaky_relu), TERMINATE]
328/328 [==============================] - 2s 6ms/step - loss: 0.0048 - root_mean_squared_error: 0.0692
Incorporated net, acc: 0.069156, net: [BILSTM(10,leaky_relu), TERMINATE]
[BILSTM(30,leaky_relu), TERMINATE]
328/328 [==============================] - 3s 8ms/step - loss: 0.0043 - root_mean_squared_error: 0.0654
Incorporated net, acc: 0.065419, net: [BILSTM(30,leaky_relu), TERMINATE]
[LSTM(10,leaky_relu), RNN(10,relu), BILSTM(30,relu), RNN(10,leaky_relu), LSTM(30,sigmoid), TERMINATE]
328/328 [==============================] - 7s 19m

## Step by Step Running

In [92]:
net, i = factory.generate_new_network()
print(net)

[RNN(100,sigmoid), GRU(100,tanh), LSTM(200,linear), FC(50, relu), TERMINATE]


In [121]:
p= netparser.parse('net', net)
newnet = netparser.parse_network_structure(p)

In [122]:
newnet

In [128]:
model = keras.Sequential(newnet)

In [213]:
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights = True)

In [129]:
model.compile(
            optimizer=keras.optimizers.Adam(lr=1e-4),
            loss='mean_squared_error',
            metrics=[keras.metrics.RootMeanSquaredError()]
        )

In [214]:
model.fit(x_train, y_train, batch_size = 40, epochs =100, callbacks=[callback], validation_data=(x_val, y_val))

1177/1177 [==============================] - 244s 208ms/step - loss: 0.0011 - root_mean_squared_error: 0.0328 - val_loss: 0.0049 - val_root_mean_squared_error: 0.0698


In [182]:
bestval = model.evaluate(x_test, y_test)[1]

328/328 [==============================] - 14s 43ms/step - loss: 0.0045 - root_mean_squared_error: 0.0672


In [61]:
factory.incorporate_trained_net(net, bestval, epsilon, [1])

Incorporated net, acc: 0.069071, net: [GRU(100,linear), BILSTM(150,relu), FC(50, leaky_relu), FC(1, leaky_relu)]


In [62]:
factory.replay_dictionary

,net,accuracy_best_val,epsilon,iteration
0,"[GRU(100,linear), BILSTM(150,relu), FC(50, lea...",0.069071,1,1


## Evaluating an architecture

In [7]:
net = "[RNN(30,tanh), D(0.200000), RNN(30,relu), RNN(30,leaky_relu), LSTM(10,leaky_relu), TERMINATE]"

In [8]:
p= netparser.parse('net', net)
newnet = netparser.parse_network_structure(p)

In [9]:
model = keras.Sequential(newnet)

In [15]:
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights = True)

In [18]:
model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=1e-4),
            loss='mean_squared_error',
            metrics=[keras.metrics.RootMeanSquaredError()]
        )

In [19]:
model.fit(x_train, y_train, batch_size = 40, epochs =100, callbacks=[callback], validation_data=(x_val, y_val))

1059/1059 [==============================] - 63s 55ms/step - loss: 0.0011 - root_mean_squared_error: 0.0335 - val_loss: 8.7292e-04 - val_root_mean_squared_error: 0.0295


In [21]:
prediction = model.predict(x_test)
prediction

In [35]:
np.savetxt('predicted_values', prediction,  fmt='%.16f')

In [22]:
model.evaluate(x_test, y_test)[1]

328/328 [==============================] - 5s 13ms/step - loss: 0.0047 - root_mean_squared_error: 0.0683


0.0683324858546257